In [1]:
import pandas as pd
import re
from collections import namedtuple
import pdfplumber
import os 
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
import numpy as np
from collections import ChainMap
import itertools
from collections import defaultdict
from operator import methodcaller
from itertools import chain

### 1. Importing and  merging pop files
### 2. Adding Metropolitans and Micropolitans

In [2]:
pop_2010_2019 = pd.read_excel('C:/Users/iulia/Documents/NSS/projects/capstone/data/raw_data/FL/pop_est_2010-2019_fl.xlsx',sheet_name='Sheet1')
pop_2010_2019 = pd.DataFrame(pop_2010_2019)
pop_2010_2019 = pop_2010_2019.drop(['Census','Estimates Base',2010,2011,2012],axis=1)
pop_2010_2019.head()

,County,2013,2014,2015,2016,2017,2018,2019
0,Alachua,252475,255456,259052,263959,266309,268851,269043
1,Baker,27009,27122,27355,27884,28254,28353,29210
2,Bay,174596,178289,181488,183634,184736,186240,174705
3,Bradford,26802,26552,26748,26740,27142,27752,28201
4,Brevard,550255,555548,565746,576874,587769,595203,601942


In [3]:
pop_2020_2021 = pd.read_excel('C:/Users/iulia/Documents/NSS/projects/capstone/data/raw_data/FL/pop_est_2020-2021_fl.xlsx',sheet_name='Sheet1')
pop_2020_2021 = pd.DataFrame(pop_2020_2021)
pop_2020_2021 = pop_2020_2021.drop('Estimates Base',axis=1)
pop_2020_2021.head()

,County,2020,2021
0,Alachua,278684,279238
1,Baker,28309,28715
2,Bay,174461,179168
3,Bradford,28364,28540
4,Brevard,608007,616628


In [4]:
pop_combined = pd.merge(pop_2010_2019,pop_2020_2021,how="left",on="County")
pop_combined.drop(index=pop_combined.index[-1],axis=0,inplace=True)
pop_combined.head()

,County,2013,2014,2015,2016,2017,2018,2019,2020,2021
0,Alachua,252475,255456,259052,263959,266309,268851,269043,278684,279238
1,Baker,27009,27122,27355,27884,28254,28353,29210,28309,28715
2,Bay,174596,178289,181488,183634,184736,186240,174705,174461,179168
3,Bradford,26802,26552,26748,26740,27142,27752,28201,28364,28540
4,Brevard,550255,555548,565746,576874,587769,595203,601942,608007,616628


In [5]:
metros = pd.read_excel('C:/Users/iulia/Documents/NSS/projects/capstone/data/metro_areas.xlsx',sheet_name='Filtered_States')
metros_df = pd.DataFrame(metros)
metros_fl = metros_df[metros_df['State Name']=='Florida']
metros_fl.head()

,CBSA Title,Metropolitan/Micropolitan Statistical Area,County,State Name
0,"Gainesville, FL",Metropolitan Statistical Area,Alachua,Florida
14,"Jacksonville, FL",Metropolitan Statistical Area,Baker,Florida
20,"Panama City, FL",Metropolitan Statistical Area,Bay,Florida
34,"Palm Bay-Melbourne-Titusville, FL",Metropolitan Statistical Area,Brevard,Florida
36,"Miami-Fort Lauderdale-Pompano Beach, FL",Metropolitan Statistical Area,Broward,Florida


### Extracting data for year 2013 and combining with pop along with Metropolitans and Micropolitans

In [6]:
file_2013 ='C:/Users/iulia/Documents/NSS/projects/capstone/data/raw_data/FL/FL_2013.pdf'

In [7]:
fl_df_crashes13 = pd.DataFrame()
pages = list(range(43,45))
pdf = pdfplumber.open(file_2013)
for page in pages:    
    table=pdf.pages[page].extract_table()
    df = pd.DataFrame(table)
    fl_df_crashes13 = fl_df_crashes13.append(df)

In [8]:
fl_df_crashes13 = fl_df_crashes13.drop([0,2,3,4,5,6,8,9,10,11],axis=1)
fl_df_crashes13 = fl_df_crashes13.iloc[:-2 , :]
fl_df_crashes13 = fl_df_crashes13.iloc[2: , :]
fl_df_crashes13.columns = ['County','total_crashes']
fl_df_crashes13.head()

,County,total_crashes
2,Alachua,"4,651"
3,Baker,293
4,Bay,"3,514"
5,Bradford,220
6,Brevard,"7,079"


In [9]:
fl_df_crashes13.tail()

,County,total_crashes
26,Unknown,0
27,Volusia,"8,047"
28,Wakulla,248
29,Walton,"1,088"
30,Washington,288


In [10]:
fl_df_fatalaties13 = pd.DataFrame()
pages = list(range(45,47))
pdf = pdfplumber.open(file_2013)
for page in pages:    
    table=pdf.pages[page].extract_table()
    df = pd.DataFrame(table)
    fl_df_fatalaties13 = fl_df_fatalaties13.append(df)

In [11]:
fl_df_fatalaties13 = fl_df_fatalaties13.drop([0,2,3,4,5,6,8,9,10,11],axis=1)
fl_df_fatalaties13 = fl_df_fatalaties13.iloc[:-2 , :]
fl_df_fatalaties13 = fl_df_fatalaties13.iloc[2: , :]
fl_df_fatalaties13.columns = ['County','fatalities']
fl_df_fatalaties13.head()

,County,fatalities
2,Alachua,33
3,Baker,9
4,Bay,17
5,Bradford,5
6,Brevard,76


In [12]:
fl_df_fatalaties13.tail()

,County,fatalities
25,Unknown,0
26,Volusia,90
27,Wakulla,3
28,Walton,17
29,Washington,8


In [13]:
fl_df_injuries13 = pd.DataFrame()
pages = list(range(47,49))
pdf = pdfplumber.open(file_2013)
for page in pages:    
    table=pdf.pages[page].extract_table()
    df = pd.DataFrame(table)
    fl_df_injuries13 = fl_df_injuries13.append(df)

In [14]:
fl_df_injuries13 = fl_df_injuries13.drop([0,2,3,4,5,6,8,9,10,11],axis=1)
fl_df_injuries13 = fl_df_injuries13.iloc[:-2 , :]
fl_df_injuries13 = fl_df_injuries13.iloc[2: , :]
fl_df_injuries13.columns = ['County','injuries']
fl_df_injuries13.head()

,County,injuries
2,Alachua,"3,321"
3,Baker,270
4,Bay,"2,208"
5,Bradford,202
6,Brevard,"5,041"


In [15]:
fl_df_injuries13.tail()

,County,injuries
25,Unknown,0
26,Volusia,"5,210"
27,Wakulla,181
28,Walton,720
29,Washington,230


In [16]:
fl_df_2013 = pd.merge(pd.merge(fl_df_fatalaties13,fl_df_injuries13,on='County'),fl_df_crashes13,on='County')
fl_df_2013 = fl_df_2013.drop(labels=63, axis=0)
fl_df_2013.head(10)

,County,fatalities,injuries,total_crashes
0,Alachua,33,"3,321","4,651"
1,Baker,9,270,293
2,Bay,17,"2,208","3,514"
3,Bradford,5,202,220
4,Brevard,76,"5,041","7,079"
5,Broward,180,"21,580","32,595"
6,Calhoun,1,109,152
7,Charlotte,22,"1,412","2,091"
8,Citrus,17,993,"1,360"
9,Clay,10,"1,621","2,434"


In [17]:
fl_df_2013.tail(10)

,County,fatalities,injuries,total_crashes
57,St. Johns,31,"2,032","2,730"
58,St. Lucie,33,"1,864","2,931"
59,Sumter,19,529,724
60,Suwannee,13,337,526
61,Taylor,6,149,198
62,Union,2,95,125
64,Volusia,90,"5,210","8,047"
65,Wakulla,3,181,248
66,Walton,17,720,"1,088"
67,Washington,8,230,288


In [18]:
df_combined_fl_13 = pd.merge(fl_df_2013,pop_combined[['County', 2013]],on="County",how='left')
df_combined_fl_2013 = pd.merge(df_combined_fl_13,metros_fl[['CBSA Title','Metropolitan/Micropolitan Statistical Area','County']],on="County",how='left')
df_combined_fl_2013.head(10)

,County,fatalities,injuries,total_crashes,2013,CBSA Title,Metropolitan/Micropolitan Statistical Area
0,Alachua,33,"3,321","4,651",252475.0,"Gainesville, FL",Metropolitan Statistical Area
1,Baker,9,270,293,27009.0,"Jacksonville, FL",Metropolitan Statistical Area
2,Bay,17,"2,208","3,514",174596.0,"Panama City, FL",Metropolitan Statistical Area
3,Bradford,5,202,220,26802.0,NaN,NaN
4,Brevard,76,"5,041","7,079",550255.0,"Palm Bay-Melbourne-Titusville, FL",Metropolitan Statistical Area
5,Broward,180,"21,580","32,595",1836600.0,"Miami-Fort Lauderdale-Pompano Beach, FL",Metropolitan Statistical Area
6,Calhoun,1,109,152,14578.0,NaN,NaN
7,Charlotte,22,"1,412","2,091",164671.0,"Punta Gorda, FL",Metropolitan Statistical Area
8,Citrus,17,993,"1,360",138829.0,"Homosassa Springs, FL",Metropolitan Statistical Area
9,Clay,10,"1,621","2,434",195657.0,"Jacksonville, FL",Metropolitan Statistical Area


In [19]:
df_combined_fl_2013.tail(10)

,County,fatalities,injuries,total_crashes,2013,CBSA Title,Metropolitan/Micropolitan Statistical Area
57,St. Johns,31,"2,032","2,730",NaN,"Jacksonville, FL",Metropolitan Statistical Area
58,St. Lucie,33,"1,864","2,931",NaN,"Port St. Lucie, FL",Metropolitan Statistical Area
59,Sumter,19,529,724,106898.0,"The Villages, FL",Metropolitan Statistical Area
60,Suwannee,13,337,526,43588.0,NaN,NaN
61,Taylor,6,149,198,22873.0,NaN,NaN
62,Union,2,95,125,15160.0,NaN,NaN
63,Volusia,90,"5,210","8,047",500438.0,"Deltona-Daytona Beach-Ormond Beach, FL",Metropolitan Statistical Area
64,Wakulla,3,181,248,31004.0,"Tallahassee, FL",Metropolitan Statistical Area
65,Walton,17,720,"1,088",59193.0,"Crestview-Fort Walton Beach-Destin, FL",Metropolitan Statistical Area
66,Washington,8,230,288,24504.0,NaN,NaN


In [20]:
df_combined_fl_2013.to_excel('C:/Users/iulia/Documents/NSS/projects/capstone/data/cleaned_data/2013_FL.xlsx', engine='xlsxwriter',index=False)

### Extracting data for years 2014 thru 2016 and combining with pop along with Metropolitans and Micropolitans

In [21]:
file_2014_2016 ='C:/Users/iulia/Documents/NSS/projects/capstone/data/raw_data/FL/FL_2016.pdf'

In [22]:
fl_df_2014_2016 = pd.DataFrame()
pages = list(range(35,42))
pdf = pdfplumber.open(file_2014_2016)
for page in pages:    
    table=pdf.pages[page].extract_table()
    table = table[1:]
    list_length = len(table)
    paired_list = [table[i] + table[i+1]+ table[i+2] for i in range(0, list_length-1,4)]
    df = pd.DataFrame(paired_list)
    fl_df_2014_2016 = fl_df_2014_2016.append(df)

In [23]:
fl_df_2014_2016 = fl_df_2014_2016.drop([5,6,7,12,13,14,19,20],axis=1)
fl_df_2014_2016.head()

,0,1,2,3,4,8,9,10,11,15,16,17,18
0,Alachua,Total Crashes,"5,980","5,027","4,193",Total Fatalities,46,37,30,Total Injuries,"4,398","3,826","3,113"
1,Baker,Total Crashes,381,373,246,Total Fatalities,13,12,2,Total Injuries,352,286,264
2,Bay,Total Crashes,"4,089","3,946","3,337",Total Fatalities,38,32,30,Total Injuries,"2,290","2,317","2,023"
3,Bradford,Total Crashes,362,348,274,Total Fatalities,6,10,4,Total Injuries,243,241,219
4,Brevard,Total Crashes,"8,752","8,387","7,661",Total Fatalities,98,82,65,Total Injuries,"6,564","6,130","5,386"


In [24]:
fl_df_2014_2016.tail()

,0,1,2,3,4,8,9,10,11,15,16,17,18
10,Union,Total Crashes,122,103,114,Total Fatalities,2,3,1,Total Injuries,84,110,131
0,Volusia,Total Crashes,"8,903","8,851","8,382",Total Fatalities,122,87,86,Total Injuries,"5,872","5,750","5,251"
1,Wakulla,Total Crashes,366,348,317,Total Fatalities,14,2,2,Total Injuries,264,261,231
2,Walton,Total Crashes,"1,305","1,248","1,127",Total Fatalities,17,9,13,Total Injuries,870,908,703
3,Washington,Total Crashes,322,349,262,Total Fatalities,5,5,3,Total Injuries,199,225,172


In [25]:
fl_df_melt_2016 = pd.melt(fl_df_2014_2016,id_vars=[2,9,16],value_vars=[0])
fl_df_melt_2016 = fl_df_melt_2016.drop(['variable'],axis=1)
fl_df_melt_2016 = fl_df_melt_2016[['value',9, 16,2]]
fl_df_melt_2016.columns = ['County','fatalaties','injuries','total_crashes']
fl_df_melt_2016.head(10)

,County,fatalaties,injuries,total_crashes
0,Alachua,46,"4,398","5,980"
1,Baker,13,352,381
2,Bay,38,"2,290","4,089"
3,Bradford,6,243,362
4,Brevard,98,"6,564","8,752"
5,Broward,245,"25,361","41,755"
6,Calhoun,3,120,194
7,Charlotte,30,"1,649","2,721"
8,Citrus,24,"1,232","1,682"
9,Clay,26,"1,857","3,053"


In [26]:
df_combined_fl_16 = pd.merge(fl_df_melt_2016,pop_combined[['County', 2016]],on="County",how='left')
df_combined_fl_2016 = pd.merge(df_combined_fl_16,metros_fl[['CBSA Title','Metropolitan/Micropolitan Statistical Area','County']],on="County",how='left')
df_combined_fl_2016.head()

,County,fatalaties,injuries,total_crashes,2016,CBSA Title,Metropolitan/Micropolitan Statistical Area
0,Alachua,46,"4,398","5,980",263959.0,"Gainesville, FL",Metropolitan Statistical Area
1,Baker,13,352,381,27884.0,"Jacksonville, FL",Metropolitan Statistical Area
2,Bay,38,"2,290","4,089",183634.0,"Panama City, FL",Metropolitan Statistical Area
3,Bradford,6,243,362,26740.0,NaN,NaN
4,Brevard,98,"6,564","8,752",576874.0,"Palm Bay-Melbourne-Titusville, FL",Metropolitan Statistical Area


In [27]:
fl_df_melt_2015 = pd.melt(fl_df_2014_2016,id_vars=[10,17,3],value_vars=[0])
fl_df_melt_2015 = fl_df_melt_2015.drop(['variable'],axis=1)
fl_df_melt_2015 = fl_df_melt_2015[['value',10,17,3]]
fl_df_melt_2015.columns = ['County','fatalaties','injuries','total_crashes']
fl_df_melt_2015.head(10)

,County,fatalaties,injuries,total_crashes
0,Alachua,37,"3,826","5,027"
1,Baker,12,286,373
2,Bay,32,"2,317","3,946"
3,Bradford,10,241,348
4,Brevard,82,"6,130","8,387"
5,Broward,221,"23,473","38,409"
6,Calhoun,4,118,180
7,Charlotte,23,"1,616","2,556"
8,Citrus,29,"1,124","1,502"
9,Clay,33,"1,727","2,817"


In [28]:
df_combined_fl_15 = pd.merge(fl_df_melt_2015,pop_combined[['County', 2015]],on="County",how='left')
df_combined_fl_2015 = pd.merge(df_combined_fl_15,metros_fl[['CBSA Title','Metropolitan/Micropolitan Statistical Area','County']],on="County",how='left')
df_combined_fl_2015.head()

,County,fatalaties,injuries,total_crashes,2015,CBSA Title,Metropolitan/Micropolitan Statistical Area
0,Alachua,37,"3,826","5,027",259052.0,"Gainesville, FL",Metropolitan Statistical Area
1,Baker,12,286,373,27355.0,"Jacksonville, FL",Metropolitan Statistical Area
2,Bay,32,"2,317","3,946",181488.0,"Panama City, FL",Metropolitan Statistical Area
3,Bradford,10,241,348,26748.0,NaN,NaN
4,Brevard,82,"6,130","8,387",565746.0,"Palm Bay-Melbourne-Titusville, FL",Metropolitan Statistical Area


In [29]:
fl_df_melt_2014 = pd.melt(fl_df_2014_2016,id_vars=[11,18,4],value_vars=[0])
fl_df_melt_2014 = fl_df_melt_2014.drop(['variable'],axis=1)
fl_df_melt_2014 = fl_df_melt_2014[['value',11,18,4]]
fl_df_melt_2014.columns = ['County','fatalaties','injuries','total_crashes']
fl_df_melt_2014.head(10)

,County,fatalaties,injuries,total_crashes
0,Alachua,30,"3,113","4,193"
1,Baker,2,264,246
2,Bay,30,"2,023","3,337"
3,Bradford,4,219,274
4,Brevard,65,"5,386","7,661"
5,Broward,173,"22,154","34,833"
6,Calhoun,2,94,134
7,Charlotte,19,"1,290","2,241"
8,Citrus,26,"1,099","1,471"
9,Clay,18,"1,784","2,594"


In [30]:
df_combined_fl_14 = pd.merge(fl_df_melt_2014,pop_combined[['County', 2014]],on="County",how='left')
df_combined_fl_2014 = pd.merge(df_combined_fl_14,metros_fl[['CBSA Title','Metropolitan/Micropolitan Statistical Area','County']],on="County",how='left')
df_combined_fl_2014.head()

,County,fatalaties,injuries,total_crashes,2014,CBSA Title,Metropolitan/Micropolitan Statistical Area
0,Alachua,30,"3,113","4,193",255456.0,"Gainesville, FL",Metropolitan Statistical Area
1,Baker,2,264,246,27122.0,"Jacksonville, FL",Metropolitan Statistical Area
2,Bay,30,"2,023","3,337",178289.0,"Panama City, FL",Metropolitan Statistical Area
3,Bradford,4,219,274,26552.0,NaN,NaN
4,Brevard,65,"5,386","7,661",555548.0,"Palm Bay-Melbourne-Titusville, FL",Metropolitan Statistical Area


In [31]:
df_combined_fl_2014.to_excel('C:/Users/iulia/Documents/NSS/projects/capstone/data/cleaned_data/2014_FL.xlsx', engine='xlsxwriter',index=False)
df_combined_fl_2015.to_excel('C:/Users/iulia/Documents/NSS/projects/capstone/data/cleaned_data/2015_FL.xlsx', engine='xlsxwriter',index=False)
df_combined_fl_2016.to_excel('C:/Users/iulia/Documents/NSS/projects/capstone/data/cleaned_data/2016_FL.xlsx', engine='xlsxwriter',index=False)

### Extracting data for year 2017

In [32]:
file_2017 ='C:/Users/iulia/Documents/NSS/projects/capstone/data/raw_data/FL/FL_2017.pdf'

In [33]:
my_dict_p35 = {}
pdf = pdfplumber.open(file_2017)
txt = pdf.pages[35].extract_text()
txt = txt[203:]
txt = txt[:-113]
county = re.findall("(\w+)(?=\\nTotal Injuries)",txt)
crashes = re.findall("(?<=Total Crashes)\s\d+\,?\d+",txt)        
fatalaties = re.findall("(?<=Total Fatalities)\s\d+",txt)          
injuries = re.findall("(?<=Total Injuries)\s\d+\,?\d+",txt)
my_dict_p35['County'] = county
my_dict_p35['fatalaties'] = fatalaties
my_dict_p35['injuries'] = injuries
my_dict_p35['total_crashes'] = crashes
print(county,crashes,fatalaties,injuries)

['Alachua', 'Baker', 'Bay', 'Bradford', 'Brevard', 'Broward', 'Calhoun', 'Charlotte', 'Citrus'] [' 6,075', ' 431', ' 4,051', ' 381', ' 9,056', ' 41,337', ' 207', ' 2,780', ' 1,781'] [' 55', ' 7', ' 29', ' 7', ' 84', ' 225', ' 1', ' 27', ' 31'] [' 4,657', ' 321', ' 2,192', ' 250', ' 6,505', ' 24,762', ' 126', ' 1,594', ' 1,382']


In [34]:
my_dict_p35['County'].append('Clay')
my_dict_p35['fatalaties'].append('21')
my_dict_p35['injuries'].append('1,804')
my_dict_p35['total_crashes'].append('2,961')
print(my_dict_p35)

{'County': ['Alachua', 'Baker', 'Bay', 'Bradford', 'Brevard', 'Broward', 'Calhoun', 'Charlotte', 'Citrus', 'Clay'], 'fatalaties': [' 55', ' 7', ' 29', ' 7', ' 84', ' 225', ' 1', ' 27', ' 31', '21'], 'injuries': [' 4,657', ' 321', ' 2,192', ' 250', ' 6,505', ' 24,762', ' 126', ' 1,594', ' 1,382', '1,804'], 'total_crashes': [' 6,075', ' 431', ' 4,051', ' 381', ' 9,056', ' 41,337', ' 207', ' 2,780', ' 1,781', '2,961']}


In [35]:
my_dict_p36 = {}
pdf = pdfplumber.open(file_2017)
txt = pdf.pages[36].extract_text()
txt = txt[120:]
txt = txt[:-70]
county = re.findall("(\w+)(?=\\nTotal Injuries)",txt)
crashes = re.findall("(?<=Total Crashes)\s\d+\,?\d+",txt)        
fatalaties = re.findall("(?<=Total Fatalities)\s\d+",txt)          
injuries = re.findall("(?<=Total Injuries)\s\d+\,?\d+",txt)
my_dict_p36['County'] = county
my_dict_p36['fatalaties'] = fatalaties
my_dict_p36['injuries'] = injuries
my_dict_p36['total_crashes'] = crashes
print(county,crashes,fatalaties,injuries)

['Collier', 'Columbia', 'DeSoto', 'Dixie', 'Duval', 'Escambia', 'Flagler', 'Franklin', 'Gadsden', 'Gilchrist'] [' 5,135', ' 1,490', ' 516', ' 151', ' 24,734', ' 6,627', ' 1,190', ' 47', ' 669', ' 189'] [' 23', ' 20', ' 15', ' 4', ' 151', ' 51', ' 33', ' 1', ' 19', ' 4'] [' 2,819', ' 1,117', ' 386', ' 120', ' 14,850', ' 4,686', ' 806', ' 35', ' 414', ' 116']


In [36]:
my_dict_p36['County'].append('Glades')
my_dict_p36['fatalaties'].append('13')
my_dict_p36['injuries'].append('153')
my_dict_p36['total_crashes'].append('202')
print(my_dict_p36)

{'County': ['Collier', 'Columbia', 'DeSoto', 'Dixie', 'Duval', 'Escambia', 'Flagler', 'Franklin', 'Gadsden', 'Gilchrist', 'Glades'], 'fatalaties': [' 23', ' 20', ' 15', ' 4', ' 151', ' 51', ' 33', ' 1', ' 19', ' 4', '13'], 'injuries': [' 2,819', ' 1,117', ' 386', ' 120', ' 14,850', ' 4,686', ' 806', ' 35', ' 414', ' 116', '153'], 'total_crashes': [' 5,135', ' 1,490', ' 516', ' 151', ' 24,734', ' 6,627', ' 1,190', ' 47', ' 669', ' 189', '202']}


In [37]:
my_dict_p37 = {}
pdf = pdfplumber.open(file_2017)
txt = pdf.pages[37].extract_text()
txt = txt[149:]
txt = txt[:-26]
county = re.findall("(\w+)(?=\\nTotal Injuries)",txt)
crashes = re.findall("(?<=Total Crashes)\s\d+\,?\d+",txt)        
fatalaties = re.findall("(?<=Total Fatalities)\s\d+",txt)          
injuries = re.findall("(?<=Total Injuries)\s\d+\,?\d+",txt)
my_dict_p37['County'] = county
my_dict_p37['fatalaties'] = fatalaties
my_dict_p37['injuries'] = injuries
my_dict_p37['total_crashes'] = crashes
print(county,crashes,fatalaties,injuries)

['Gulf', 'Hamilton', 'Hardee', 'Hendry', 'Hernando', 'Highlands', 'Hillsborough', 'Holmes', 'River', 'Jackson'] [' 183', ' 321', ' 476', ' 405', ' 2,312', ' 1,367', ' 27,909', ' 254', ' 1,728', ' 814'] [' 9', ' 9', ' 12', ' 9', ' 34', ' 18', ' 191', ' 10', ' 28', ' 18'] [' 97', ' 211', ' 408', ' 302', ' 2,178', ' 958', ' 20,682', ' 179', ' 1,079', ' 474']


In [38]:
print(my_dict_p37)

{'County': ['Gulf', 'Hamilton', 'Hardee', 'Hendry', 'Hernando', 'Highlands', 'Hillsborough', 'Holmes', 'River', 'Jackson'], 'fatalaties': [' 9', ' 9', ' 12', ' 9', ' 34', ' 18', ' 191', ' 10', ' 28', ' 18'], 'injuries': [' 97', ' 211', ' 408', ' 302', ' 2,178', ' 958', ' 20,682', ' 179', ' 1,079', ' 474'], 'total_crashes': [' 183', ' 321', ' 476', ' 405', ' 2,312', ' 1,367', ' 27,909', ' 254', ' 1,728', ' 814']}


In [39]:
my_dict_p38 = {}
pdf = pdfplumber.open(file_2017)
txt = pdf.pages[38].extract_text()
txt = txt[68:]
txt = txt[:-161]
county = re.findall("(\w+)(?=\\nTotal Injuries)",txt)
crashes = re.findall("(?<=Total Crashes)\s\d+\,?\d+",txt)        
fatalaties = re.findall("(?<=Total Fatalities)\s\d+",txt)          
injuries = re.findall("(?<=Total Injuries)\s\d+\,?\d+",txt)
my_dict_p38['County'] = county
my_dict_p38['fatalaties'] = fatalaties
my_dict_p38['injuries'] = injuries
my_dict_p38['total_crashes'] = crashes
print(county,crashes,fatalaties,injuries)

['Jefferson', 'Lafayette', 'Lake', 'Lee', 'Leon', 'Levy', 'Liberty', 'Madison', 'Manatee', 'Marion'] [' 264', ' 35', ' 4,731', ' 12,709', ' 7,113', ' 584', ' 99', ' 350', ' 5,973', ' 6,691'] [' 4', ' 5', ' 48', ' 113', ' 34', ' 23', ' 3', ' 2', ' 54', ' 90'] [' 133', ' 39', ' 3,627', ' 7,832', ' 3,982', ' 449', ' 74', ' 201', ' 4,610', ' 5,135']


In [40]:
my_dict_p38['County'].append('Martin')
my_dict_p38['fatalaties'].append('25')
my_dict_p38['injuries'].append('2,087')
my_dict_p38['total_crashes'].append('3,466')
print(my_dict_p38)

{'County': ['Jefferson', 'Lafayette', 'Lake', 'Lee', 'Leon', 'Levy', 'Liberty', 'Madison', 'Manatee', 'Marion', 'Martin'], 'fatalaties': [' 4', ' 5', ' 48', ' 113', ' 34', ' 23', ' 3', ' 2', ' 54', ' 90', '25'], 'injuries': [' 133', ' 39', ' 3,627', ' 7,832', ' 3,982', ' 449', ' 74', ' 201', ' 4,610', ' 5,135', '2,087'], 'total_crashes': [' 264', ' 35', ' 4,731', ' 12,709', ' 7,113', ' 584', ' 99', ' 350', ' 5,973', ' 6,691', '3,466']}


In [41]:
my_dict_p39 = {}
pdf = pdfplumber.open(file_2017)
txt = pdf.pages[39].extract_text()
txt = txt[75:]
txt = txt[:-120]
county = re.findall("(\w+\-?\w+?)(?=\\nTotal Injuries)",txt)
crashes = re.findall("(?<=Total Crashes)\s\d+\,?\d+",txt)        
fatalaties = re.findall("(?<=Total Fatalities)\s\d+",txt)          
injuries = re.findall("(?<=Total Injuries)\s\d+\,?\d+",txt)
my_dict_p39['County'] = county
my_dict_p39['fatalaties'] = fatalaties
my_dict_p39['injuries'] = injuries
my_dict_p39['total_crashes'] = crashes
print(county,crashes,fatalaties,injuries)

['Miami-Dade', 'Monroe', 'Nassau', 'Okaloosa', 'Okeechobee', 'Orange', 'Osceola', 'Beach', 'Pasco', 'Pinellas'] [' 65,986', ' 1,255', ' 1,129', ' 3,527', ' 719', ' 31,004', ' 6,494', ' 27,273', ' 7,793', ' 17,968'] [' 285', ' 18', ' 18', ' 32', ' 13', ' 188', ' 55', ' 162', ' 107', ' 119'] [' 32,389', ' 650', ' 825', ' 2,172', ' 507', ' 21,646', ' 5,436', ' 15,912', ' 6,382', ' 11,107']


In [42]:
my_dict_p39['County'].append('Polk')
my_dict_p39['fatalaties'].append('112')
my_dict_p39['injuries'].append('7,555')
my_dict_p39['total_crashes'].append('11,378')
print(my_dict_p39)

{'County': ['Miami-Dade', 'Monroe', 'Nassau', 'Okaloosa', 'Okeechobee', 'Orange', 'Osceola', 'Beach', 'Pasco', 'Pinellas', 'Polk'], 'fatalaties': [' 285', ' 18', ' 18', ' 32', ' 13', ' 188', ' 55', ' 162', ' 107', ' 119', '112'], 'injuries': [' 32,389', ' 650', ' 825', ' 2,172', ' 507', ' 21,646', ' 5,436', ' 15,912', ' 6,382', ' 11,107', '7,555'], 'total_crashes': [' 65,986', ' 1,255', ' 1,129', ' 3,527', ' 719', ' 31,004', ' 6,494', ' 27,273', ' 7,793', ' 17,968', '11,378']}


In [43]:
my_dict_p40 = {}
pdf = pdfplumber.open(file_2017)
txt = pdf.pages[40].extract_text()
txt = txt[120:]
txt = txt[:-78]
county = re.findall("(\w+\-?\w+?)(?=\\nTotal Injuries)",txt)
crashes = re.findall("(?<=Total Crashes)\s\d+\,?\d+",txt)        
fatalaties = re.findall("(?<=Total Fatalities)\s\d+",txt)          
injuries = re.findall("(?<=Total Injuries)\s\d+\,?\d+",txt)
my_dict_p40['County'] = county
my_dict_p40['fatalaties'] = fatalaties
my_dict_p40['injuries'] = injuries
my_dict_p40['total_crashes'] = crashes
print(county,crashes,fatalaties,injuries)

['Putnam', 'Rosa', 'Sarasota', 'Seminole', 'Johns', 'Lucie', 'Sumter', 'Suwannee', 'Taylor', 'Union'] [' 1,210', ' 1,893', ' 6,625', ' 7,211', ' 3,549', ' 5,062', ' 1,650', ' 713', ' 325', ' 136'] [' 26', ' 19', ' 49', ' 48', ' 42', ' 41', ' 23', ' 14', ' 13', ' 4'] [' 902', ' 1,597', ' 4,002', ' 4,660', ' 2,113', ' 3,093', ' 983', ' 451', ' 209', ' 125']


In [44]:
my_dict_p40['County'].append('Volusia')
my_dict_p40['fatalaties'].append('130')
my_dict_p40['injuries'].append('6,527')
my_dict_p40['total_crashes'].append('9,601')
print(my_dict_p40)

{'County': ['Putnam', 'Rosa', 'Sarasota', 'Seminole', 'Johns', 'Lucie', 'Sumter', 'Suwannee', 'Taylor', 'Union', 'Volusia'], 'fatalaties': [' 26', ' 19', ' 49', ' 48', ' 42', ' 41', ' 23', ' 14', ' 13', ' 4', '130'], 'injuries': [' 902', ' 1,597', ' 4,002', ' 4,660', ' 2,113', ' 3,093', ' 983', ' 451', ' 209', ' 125', '6,527'], 'total_crashes': [' 1,210', ' 1,893', ' 6,625', ' 7,211', ' 3,549', ' 5,062', ' 1,650', ' 713', ' 325', ' 136', '9,601']}


In [45]:
my_dict_p41 = {}
pdf = pdfplumber.open(file_2017)
txt = pdf.pages[41].extract_text()
txt = txt[160:]
txt = txt[:-26]
county = re.findall("(\w+\-?\w+?)(?=\\nTotal Injuries)",txt)
crashes = re.findall("(?<=Total Crashes)\s\d+\,?\d+",txt)        
fatalaties = re.findall("(?<=Total Fatalities)\s\d+",txt)          
injuries = re.findall("(?<=Total Injuries)\s\d+\,?\d+",txt)
my_dict_p41['County'] = county
my_dict_p41['fatalaties'] = fatalaties
my_dict_p41['injuries'] = injuries
my_dict_p41['total_crashes'] = crashes
print(county,crashes,fatalaties,injuries)

['Wakulla', 'Walton', 'Washington'] [' 432', ' 1,300', ' 358'] [' 7', ' 21', ' 10'] [' 302', ' 748', ' 185']


In [46]:
my_dict_combined = defaultdict(list)
dict_items = map(methodcaller('items'), (my_dict_p35,my_dict_p36,my_dict_p37,my_dict_p38,my_dict_p39,my_dict_p40,my_dict_p41))
for k, v in chain.from_iterable(dict_items):
    my_dict_combined[k].extend(v)

print(my_dict_combined)

defaultdict(<class 'list'>, {'County': ['Alachua', 'Baker', 'Bay', 'Bradford', 'Brevard', 'Broward', 'Calhoun', 'Charlotte', 'Citrus', 'Clay', 'Collier', 'Columbia', 'DeSoto', 'Dixie', 'Duval', 'Escambia', 'Flagler', 'Franklin', 'Gadsden', 'Gilchrist', 'Glades', 'Gulf', 'Hamilton', 'Hardee', 'Hendry', 'Hernando', 'Highlands', 'Hillsborough', 'Holmes', 'River', 'Jackson', 'Jefferson', 'Lafayette', 'Lake', 'Lee', 'Leon', 'Levy', 'Liberty', 'Madison', 'Manatee', 'Marion', 'Martin', 'Miami-Dade', 'Monroe', 'Nassau', 'Okaloosa', 'Okeechobee', 'Orange', 'Osceola', 'Beach', 'Pasco', 'Pinellas', 'Polk', 'Putnam', 'Rosa', 'Sarasota', 'Seminole', 'Johns', 'Lucie', 'Sumter', 'Suwannee', 'Taylor', 'Union', 'Volusia', 'Wakulla', 'Walton', 'Washington'], 'fatalaties': [' 55', ' 7', ' 29', ' 7', ' 84', ' 225', ' 1', ' 27', ' 31', '21', ' 23', ' 20', ' 15', ' 4', ' 151', ' 51', ' 33', ' 1', ' 19', ' 4', '13', ' 9', ' 9', ' 12', ' 9', ' 34', ' 18', ' 191', ' 10', ' 28', ' 18', ' 4', ' 5', ' 48', ' 113'

In [47]:
fl_df_2017 = pd.DataFrame(my_dict_combined)
fl_df_2017.head()

,County,fatalaties,injuries,total_crashes
0,Alachua,55,"4,657","6,075"
1,Baker,7,321,431
2,Bay,29,"2,192","4,051"
3,Bradford,7,250,381
4,Brevard,84,"6,505","9,056"


In [48]:
fl_df_2017.tail()

,County,fatalaties,injuries,total_crashes
62,Union,4,125,136
63,Volusia,130,"6,527","9,601"
64,Wakulla,7,302,432
65,Walton,21,748,"1,300"
66,Washington,10,185,358


In [49]:
df_combined_fl_17 = pd.merge(fl_df_2017,pop_combined[['County', 2017]],on="County",how='left')
df_combined_fl_2017 = pd.merge(df_combined_fl_17,metros_fl[['CBSA Title','Metropolitan/Micropolitan Statistical Area','County']],on="County",how='left')
df_combined_fl_2017.head()

,County,fatalaties,injuries,total_crashes,2017,CBSA Title,Metropolitan/Micropolitan Statistical Area
0,Alachua,55,"4,657","6,075",266309.0,"Gainesville, FL",Metropolitan Statistical Area
1,Baker,7,321,431,28254.0,"Jacksonville, FL",Metropolitan Statistical Area
2,Bay,29,"2,192","4,051",184736.0,"Panama City, FL",Metropolitan Statistical Area
3,Bradford,7,250,381,27142.0,NaN,NaN
4,Brevard,84,"6,505","9,056",587769.0,"Palm Bay-Melbourne-Titusville, FL",Metropolitan Statistical Area


In [50]:
df_combined_fl_2017.to_excel('C:/Users/iulia/Documents/NSS/projects/capstone/data/cleaned_data/2017_FL.xlsx', engine='xlsxwriter',index=False)

### Extracting data for years 2018 thru 2020 and combining with pop along with Metropolitans and Micropolitans

In [51]:
file_2018_2020 ='C:/Users/iulia/Documents/NSS/projects/capstone/data/raw_data/FL/FL_2020.pdf'

In [52]:
fl_df_2018_2020 = pd.DataFrame()
pages = list(range(38,46))
pdf = pdfplumber.open(file_2018_2020)
for page in pages:    
    table=pdf.pages[page].extract_table()
    table = table[1:]
    list_length = len(table)
    paired_list = [table[i] + table[i+1]+ table[i+2] for i in range(0, list_length-1,4)]
    df = pd.DataFrame(paired_list)
    fl_df_2018_2020 = fl_df_2018_2020.append(df)

In [53]:
fl_df_2018_2020 = fl_df_2018_2020.drop([5,6,7,12,13,14,19,20],axis=1)
fl_df_2018_2020 = fl_df_2018_2020.iloc[:-1 , :]
fl_df_2018_2020.head()

,0,1,2,3,4,8,9,10,11,15,16,17,18
0,Alachua,Total Crashes,"4,617","5,878","5,846",Total Fatalities,48,72,51,Total Injuries,"3,266","4,237","4,415"
1,Baker,Total Crashes,420,438,400,Total Fatalities,4,7,5,Total Injuries,293,341,354
2,Bay,Total Crashes,"3,352","4,862","4,549",Total Fatalities,35,32,26,Total Injuries,"1,869","2,498","2,360"
3,Bradford,Total Crashes,329,412,449,Total Fatalities,10,12,9,Total Injuries,192,282,252
4,Brevard,Total Crashes,"8,395","9,207","9,233",Total Fatalities,81,84,85,Total Injuries,"5,768","6,625","6,911"


In [54]:
fl_df_2018_2020.tail()

,0,1,2,3,4,8,9,10,11,15,16,17,18
8,Union,Total Crashes,144,152,161,Total Fatalities,2,3,3,Total Injuries,135,117,142
0,Volusia,Total Crashes,"9,141","10,133","9,967",Total Fatalities,137,126,106,Total Injuries,"6,271","6,817","6,657"
1,Wakulla,Total Crashes,407,412,422,Total Fatalities,3,8,9,Total Injuries,215,262,301
2,Walton,Total Crashes,"1,342","1,383","1,412",Total Fatalities,29,19,24,Total Injuries,703,836,822
3,Washington,Total Crashes,432,360,348,Total Fatalities,10,9,10,Total Injuries,186,227,203


In [55]:
fl_df_melt_2020 = pd.melt(fl_df_2018_2020,id_vars=[2,9,16],value_vars=[0])
fl_df_melt_2020 = fl_df_melt_2020.drop(['variable'],axis=1)
fl_df_melt_2020 = fl_df_melt_2020[['value',9, 16,2]]
fl_df_melt_2020.columns = ['County','fatalaties','injuries','total_crashes']
fl_df_melt_2020.head(10)

,County,fatalaties,injuries,total_crashes
0,Alachua,48,"3,266","4,617"
1,Baker,4,293,420
2,Bay,35,"1,869","3,352"
3,Bradford,10,192,329
4,Brevard,81,"5,768","8,395"
5,Broward,268,"18,984","34,050"
6,Calhoun,3,120,198
7,Charlotte,26,"1,369","2,185"
8,Citrus,38,"1,194","1,685"
9,Clay,24,"1,502","2,511"


In [56]:
df_combined_fl_20 = pd.merge(fl_df_melt_2020,pop_combined[['County', 2020]],on="County",how='left')
df_combined_fl_2020 = pd.merge(df_combined_fl_20,metros_fl[['CBSA Title','Metropolitan/Micropolitan Statistical Area','County']],on="County",how='left')
df_combined_fl_2020.head()

,County,fatalaties,injuries,total_crashes,2020,CBSA Title,Metropolitan/Micropolitan Statistical Area
0,Alachua,48,"3,266","4,617",278684.0,"Gainesville, FL",Metropolitan Statistical Area
1,Baker,4,293,420,28309.0,"Jacksonville, FL",Metropolitan Statistical Area
2,Bay,35,"1,869","3,352",174461.0,"Panama City, FL",Metropolitan Statistical Area
3,Bradford,10,192,329,28364.0,NaN,NaN
4,Brevard,81,"5,768","8,395",608007.0,"Palm Bay-Melbourne-Titusville, FL",Metropolitan Statistical Area


In [57]:
fl_df_melt_2019 = pd.melt(fl_df_2018_2020,id_vars=[10,17,3],value_vars=[0])
fl_df_melt_2019 = fl_df_melt_2019.drop(['variable'],axis=1)
fl_df_melt_2019 = fl_df_melt_2019[['value',10,17,3]]
fl_df_melt_2019.columns = ['County','fatalaties','injuries','total_crashes']
fl_df_melt_2019.head(10)

,County,fatalaties,injuries,total_crashes
0,Alachua,72,"4,237","5,878"
1,Baker,7,341,438
2,Bay,32,"2,498","4,862"
3,Bradford,12,282,412
4,Brevard,84,"6,625","9,207"
5,Broward,214,"23,671","41,114"
6,Calhoun,4,148,213
7,Charlotte,22,"1,418","2,386"
8,Citrus,22,"1,410","1,774"
9,Clay,26,"1,669","2,694"


In [58]:
df_combined_fl_19 = pd.merge(fl_df_melt_2019,pop_combined[['County', 2019]],on="County",how='left')
df_combined_fl_2019 = pd.merge(df_combined_fl_19,metros_fl[['CBSA Title','Metropolitan/Micropolitan Statistical Area','County']],on="County",how='left')
df_combined_fl_2019.head()

,County,fatalaties,injuries,total_crashes,2019,CBSA Title,Metropolitan/Micropolitan Statistical Area
0,Alachua,72,"4,237","5,878",269043.0,"Gainesville, FL",Metropolitan Statistical Area
1,Baker,7,341,438,29210.0,"Jacksonville, FL",Metropolitan Statistical Area
2,Bay,32,"2,498","4,862",174705.0,"Panama City, FL",Metropolitan Statistical Area
3,Bradford,12,282,412,28201.0,NaN,NaN
4,Brevard,84,"6,625","9,207",601942.0,"Palm Bay-Melbourne-Titusville, FL",Metropolitan Statistical Area


In [59]:
fl_df_melt_2018 = pd.melt(fl_df_2018_2020,id_vars=[11,18,4],value_vars=[0])
fl_df_melt_2018 = fl_df_melt_2018.drop(['variable'],axis=1)
fl_df_melt_2018 = fl_df_melt_2018[['value',11,18,4]]
fl_df_melt_2018.columns = ['County','fatalaties','injuries','total_crashes']
fl_df_melt_2018.head(10)

,County,fatalaties,injuries,total_crashes
0,Alachua,51,"4,415","5,846"
1,Baker,5,354,400
2,Bay,26,"2,360","4,549"
3,Bradford,9,252,449
4,Brevard,85,"6,911","9,233"
5,Broward,237,"24,644","41,082"
6,Calhoun,8,112,206
7,Charlotte,27,"1,582","2,734"
8,Citrus,37,"1,413","1,813"
9,Clay,19,"1,737","2,849"


In [60]:
df_combined_fl_18 = pd.merge(fl_df_melt_2018,pop_combined[['County', 2018]],on="County",how='left')
df_combined_fl_2018 = pd.merge(df_combined_fl_18,metros_fl[['CBSA Title','Metropolitan/Micropolitan Statistical Area','County']],on="County",how='left')
df_combined_fl_2018.head()

,County,fatalaties,injuries,total_crashes,2018,CBSA Title,Metropolitan/Micropolitan Statistical Area
0,Alachua,51,"4,415","5,846",268851.0,"Gainesville, FL",Metropolitan Statistical Area
1,Baker,5,354,400,28353.0,"Jacksonville, FL",Metropolitan Statistical Area
2,Bay,26,"2,360","4,549",186240.0,"Panama City, FL",Metropolitan Statistical Area
3,Bradford,9,252,449,27752.0,NaN,NaN
4,Brevard,85,"6,911","9,233",595203.0,"Palm Bay-Melbourne-Titusville, FL",Metropolitan Statistical Area


In [61]:
df_combined_fl_2018.to_excel('C:/Users/iulia/Documents/NSS/projects/capstone/data/cleaned_data/2018_FL.xlsx', engine='xlsxwriter',index=False)
df_combined_fl_2019.to_excel('C:/Users/iulia/Documents/NSS/projects/capstone/data/cleaned_data/2019_FL.xlsx', engine='xlsxwriter',index=False)
df_combined_fl_2020.to_excel('C:/Users/iulia/Documents/NSS/projects/capstone/data/cleaned_data/2020_FL.xlsx', engine='xlsxwriter',index=False)